# Diamond characteristics as predictors of diamond pricing

## Introduction

Diamonds dataset by ggplot2 describes different diamond characteristics and diamond prices. It's crucial for people in the diamond business and customers to be able to understand the correct pricing of diamonds. Understanding the best predictors of pricing allow people to ensure they don't get scammed in the business. It also allows us to create models which predict the correct pricing range of the diamonds.

We would like to know which dimensional characteristics of diamond have the greatest effect on diamond pricing. To test the relevance of variables to the pricing we are going to employ different variable selection methods such as selection using p-values and LASSO analysis.

Let's begin by loading the data and important libraries.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML, display
import tabulate
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'tabulate'

## Analysing data

### Variables

Let us start by exploring contents of the data.

In [ ]:
df = pd.read_csv('diamonds.csv')
df.head()

Data contains information about diamonds
* carat: which is unit used to describe weight of diamond
* cut: how well diamond has been cut
* color: coloring of diamond
* clarity: measurement how clear diamond is (I1 (worst), SI2, SI1, VS2, VS1, VVS2, VVS1, IF (best))
* depth: depth presentage based on dimensions of x, y, z
* table: width of the diamond relative to widest point
* price: pricing of the diamond in US dollars
* x, y, z: dimensions of diamond: length, width and depth respectively

In [ ]:
#distributional data labels
num_labels = ['carat', 'x', 'y', 'z', 'depth', 'price', 'table']
#categorial data labels
feat_labels = ['cut', 'color', 'clarity']

#### Distributional data

To understand distributional data we first want to analyze its descriptive statistics. Let us first create a table.

In [ ]:
table = [["","mean","median","1. quantile"
          , "3. quantile", "std", "minimum", "maximum", "skewness"]]
for i in range(len(num_labels)):
    data = df[num_labels[i]].values
    table.append([num_labels[i]])
    table[i + 1].extend([
        np.mean(data),
        np.median(data),
        np.percentile(data, 25),
        np.percentile(data, 75),
        np.sqrt(np.var(data)),
        np.min(data),
        np.max(data),
        (np.mean((data - data.mean()) ** 3))/ (np.std(data, ddof = 1) ** 3)
    ])
display(HTML(tabulate.tabulate(table, tablefmt='html')))

We notice that all variables but depth seem to be heavily skewed to right. This is expected due to all of the variables being non-negative characteristics of the diamond. Also, the variance is kinda high for distributions to resemble a single normal distribution. Something else must be in play.

Depth seems to form simple unimodal probability distribution since the quantiles seem to be equally distributed and the skewness is almost zero.

In [ ]:
# function for removing the outliers from data
def robust_data(data, outlier_persent):
    lower = np.partition(data, int(data.size * outlier_persent))
    lower = lower[int(data.size * outlier_persent)]
    upper = np.partition(data, int(data.size * (1 - outlier_persent)))
    upper = upper[int(data.size * (1 - outlier_persent))]
    return data[(data > lower) & (data < upper)]

def robust_pair(feat, label, outlier_persent):
    lower = np.partition(feat, int(feat.size * outlier_persent))
    lower = lower[int(feat.size * outlier_persent)]
    upper = np.partition(feat, int(feat.size * (1 - outlier_persent)))
    upper = upper[int(feat.size * (1 - outlier_persent))]
    return feat[(feat > lower) & (feat < upper)] , label[(feat > lower) & (feat < upper)]

In [ ]:
for label in num_labels:
    data = df[label].values
    data = robust_data(data, 0.001)
    plt.hist(data, bins = 'rice', range = [np.min(data), np.max(data)] )
    plt.title(label + ' distribution')
    plt.show()

Carat and dimensional x, y and z variables seem to form distributions alike. Each of them has local spikes similarly distributed. This is likely due to diamonds being refined into commonly used sizes. Also for each of them, spikes on the right side get smaller and seem to form a right-leaning tail.

Depth actually isn't unimodal. It has many spikes at integer values due to rounding of the values. Still, the integer values seem to form binomial distribution and the other data closely resembles a normal distribution.

Table values seem to be a discrete variable. However, there exists values between the integers. This is likely due to table values being rounded.

In [ ]:
for label in feat_labels:
    unique, counts = np.unique(df[label].values, return_counts=True)
    ind = np.arange(len(unique))
    plt.bar(ind,counts)
    plt.xticks(ind, unique)
    plt.show()

The only significant attention must be given to low quantity of boft only fairly cut diamonds and to the dimmest of diamonds.

In [ ]:
for label in num_labels:
    if label != 'price':
        data = df[label].values
        data, price = robust_pair(data, df['price'].values, 0.001)
        plt.scatter(data, price, s = 0.1)
        plt.xlabel(label)
        plt.show()

Carat, x, y and z variables seem to have a positive dependency on the price. Because of the distribution of the data, it is hard to see if there exist any trends in depth or table.

For curiosity let's also look at categorical variables.

In [ ]:
for label in feat_labels:
    sns.boxplot(x = label, y = 'price', data = df)
    plt.show()

Each categorical variable seems to have really long tails on high price size and they seem to have only a minor effect on the price.

# Variable selection

We are going to perform a variable selection for predicting diamond pricing. Our hypothesis is that carat of the diamond has a high effect on diamond pricing and the dimensional variables are completely redundant since the information about diamond size is included in carat. We will first examine dimensional variables of the diamond using a linear model.

Let us first define the linear model for predicting the price of diamonds.

$price_i = \boldsymbol{\beta}^T \boldsymbol{x}_i + \beta_0 $

Where $ \boldsymbol{\beta}$ represents the coefficient vector for the model and $\beta_0$ represents the constant term.

We will perform a backward selection and LASSO analysis for the variables. For backward selection, our hypothesis means that carat should be the last variable left for the model as we reduce variables in order of highest p-value. For LASSO analysis carat should be the last variable to reach the zero as we increase the $\alpha$ penalty of coefficients.

Our models assume that the variables have a linear dependence on the price and that they are linearly independent. This might not be the case and we will examine it's implications later. Also, we assume that the variance of all variables is the same. To ensure this we are going to use a standard scaler to standardize the features. This is done by taking the difference of datapoint and mean and dividing it by standard deviation [3].

$x_{standard} = (x - \bar{x})/\sigma$

## Backward selection

Lets fit the linear model to data and analyze the p-values of variables.

In [ ]:
price = df['price'].values
labels = ['carat', 'depth', 'table', 'x', 'y', 'z']
num_data = df[labels].values

In [ ]:
def calculate_p(x, y, y_hat, model_params):
    tmp = np.hstack((np.ones((x.shape[0],1)),x))
    MSE = sum((y - y_hat) ** 2) / (len(tmp) - len(tmp[0]))
    std = np.sqrt(MSE * (np.linalg.inv(np.dot(tmp.T,tmp)).diagonal()))
    ts = model_params / std
    p_vals = [2 * (1 - stats.t.cdf(np.abs(i),(len(tmp)-1))) for i in ts]
    return p_vals

In [ ]:
# this is for testing purposes
def regress(lab, dp):
    lm = LinearRegression()
    data = df[lab].values
    x_train, x_test, y_train, y_test = train_test_split(data,price,test_size = dp, random_state = 123)
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_train)
    lm.fit(x_train, y_train)
    predict_data = lm.predict(x_train)
    p_vals = calculate_p(x_train, y_train, predict_data, np.append(lm.intercept_,lm.coef_))
    return p_vals

def print_table(p_vals, lab):
    table = [["","p-values"]]
    for i in range(len(lab)):
        table.append([lab[i], p_vals[i + 1]])
    display(HTML(tabulate.tabulate(table, tablefmt='html')))

def exe(lab, dp):
    print_table(regress(lab, dp),lab)

Lets start with normal backwards selection with small $\alpha = 0.01$

In [ ]:
labels = ['carat', 'depth', 'table', 'x', 'y', 'z']
exe(labels, 0)

In [ ]:
labels = ['carat', 'depth', 'table', 'x', 'y']
exe(labels, 0)

Now all the values are above chosen alpha. This would be the chosen model. Because we wanted to find the best predictors for the price we will continue reducing variables. Because of 64-bit floats aren't accurate enough to capture such a small p-value we will take a percent sample from the dataset. This isn't a reliable method for finding the best variables, but it should indicate what the result should be.

In [ ]:
labels = ['carat', 'depth', 'table', 'x', 'y', 'z']
exe(labels, 0.99)

In [ ]:
labels = ['carat', 'table', 'x', 'y', 'z']
exe(labels, 0.99)

In [ ]:
labels = ['carat', 'x', 'y', 'z']
exe(labels, 0.99)

In [ ]:
labels = ['carat', 'y', 'z']
exe(labels, 0.99)

In [ ]:
labels = ['carat', 'z']
exe(labels, 0.99)

In [ ]:
labels = ['carat']
exe(labels, 0.99)

# LASSO

In LASSO we create a linear predictor. However to make it more robust we penalize the coefficients. Instead of just minimizing the residual we also try to minimize the coefficients. To achieve this we formulate a new cost function.

$C = ||y - Xw||^2_2 + \alpha \cdot ||w||_1$

where $\alpha$ is the penalization coefficient

In [ ]:
labels = ['carat','depth', 'table', 'x', 'y', 'z']
X = df[labels].values
y = df['price'].values
col = []
alpha = []
X = StandardScaler().fit_transform(X)
placeholder = 0.1
alphas = np.arange(0.1,3000,0.1)
model = Lasso(placeholder, warm_start = True)
for i in alphas:
    model.set_params(alpha = i)
    alpha.append(np.log(i))
    model.fit(X,y)
    col.append(np.log(np.abs(model.coef_) + 1))
plt.plot(alpha, col)
plt.legend(labels)
plt.show()

The penalty increases on a logarithmic scale from left to right. The least important variables reach the zero first and the most important last.

Unlike the p-values, LASSO seems to value the relevance of depth and table more. This might be due to table and depth being less linearly dependent, but with less covariance than x, y and z. This would also explain why those measurements are taken in the first place.

Let us find the best values of alpha using cross-validation.

In [ ]:
X = df[labels].values #back to regular data to avoid data leakage
model = LassoCV(cv = 10, alphas = alphas, normalize = True).fit(X,y)
print(model.alpha_)
model = LassoCV(cv = 10, alphas = np.arange(0.001,0.01,0.001), normalize = True).fit(X,y)
print(model.alpha_)
model = LassoCV(cv = 10, alphas = np.arange(0.0001,0.001,0.0001), normalize = True).fit(X,y)
print(model.alpha_)
model = LassoCV(cv = 10, alphas = np.arange(0.000001,0.00001,0.000001), normalize = True).fit(X,y)
print(model.alpha_)
print("Carat coef = " + str(model.coef_[0]))

Although it seems that leaving x,y and z variables might be a good method for reducing explanatory variables it seems that they are useful at predicting the prices.

Let us analyze our assumptions of data being uncorrelated.

In [ ]:
labels = ['carat', 'depth', 'table', 'x', 'y', 'z']
corr = df[labels].corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(2)

It seems that x,y,z and carat of the diamond are highly correlated. However, that's not a problem since LASSO is often robust at selecting the best variables then the distinction between the relevance of variables is noticeable. Carat was found to be the most reliable predictor also by the backward selection so we can rely on our results.

# Conclusions

It seems that our hypothesis of carat being the most important dimensional predictor of the diamond pricing was correct. However, it seems that the prediction of other dimensions being redundant wasn't true. LASSO analysis showed that depth and table of the diamonds seem to be quite important. More importantly, LASSO cross-validation showed that the best model is obtained without the penalization. This means that all variables should be important for predicting the price of the diamond.

Using statistical models we have compared dimensional properties of the diamond such as carat, length, width, depth, the width of the diamond relative to the widest point and the depth percentage of the diamond. We found that all variables are valid predictors of the diamond price. It seems though that carat of the diamond is superior at predicting the value of the diamond. Width relative to the widest point and depth percentage were also found to be quite useful.

The analysis could be continued by trying non-linear models. Then the variable selection would have had to be done by more efficient methods such as Branch-and-Bound algorithms. Another alternative method could have been least-angle regression. We could also try to divide prices into bins and use gradient boosting algorithm for finding the relevance of different variables.

Also, it would be interesting to know how important categorical variables such as cut, color, and clarity are for pricing of diamonds.

## References

[1] R-documentation, diamond prices, https://vincentarelbundock.github.io/Rdatasets/doc/ggplot2/diamonds.html

[2] Sklearn-documentation, LASSO, https://scikit-learn.org/stable/auto_examples/linear_model/plot_lasso_lars.html

[3] Sklearn-documentation , StandardScaler, https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html